In [1]:
import pandas as pd
import numpy as np
from itertools import chain
df = pd.read_csv('HMAHCC_COMP.csv')

c:\users\chenson\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
opioid_data = df[df['PAY_DAY_SUPPLY_CNT'].notnull()]
opioid_data_grouped = opioid_data.groupby(by='id')

In [3]:
def get_LTOT_frame(ID):
    try:
        group = opioid_data_grouped.get_group(ID)
        frame = group[['Days', 'PAY_DAY_SUPPLY_CNT']].drop_duplicates()
        frame = frame[frame['Days']>=0]
        frame['drugs until'] = (frame['Days'] + frame['PAY_DAY_SUPPLY_CNT']).astype(int)
        frame['range'] = frame.apply(lambda x : range(x['Days'].astype(int),x['drugs until'].astype(int)),1)

        concat = concatenated = chain(*list(frame['range']))
        concat = set(concat)

        day_frame = pd.DataFrame(columns = ['Days', 'Has Drug?'])
        day_frame['Days'] = range(max(concat)+1)
        day_frame['Has Drug?'] = (day_frame['Days'].isin(concat))
        
        return day_frame
    except:
        return "No data"

In [147]:
def naive(ID):
    try:
        frame = get_LTOT_frame(ID)
        frame['naive'] = frame['Has Drug?'].rolling(90, win_type=None).sum()
        qualify = frame[ (frame['naive'] == 1) & (frame['Has Drug?'] == True)]
        qualify_list = [0] + list(qualify['Days'].values)
        return(qualify_list)
    except:
        return(np.NaN)

In [122]:
id_list = df['id'].drop_duplicates().values
id_list

array(['ID10010854159', 'ID10013863216', 'ID10024447278', ...,
       'ID99985278397', 'ID99987501651', 'ID99994197161'], dtype=object)

In [149]:
%%time
response_variable_v2 = pd.DataFrame(id_list, columns = ['id'])
response_variable_v2['naive'] = response_variable_v2['id'].map(naive)

Wall time: 48min 26s


In [168]:
response_variable_v2

,id,naive
0,ID10010854159,"[0, 706]"
1,ID10013863216,[0]
2,ID10024447278,[0]
3,ID1002482139,[0]
4,ID1003386406,[0]
5,ID10036289882,[0]
6,ID10036954381,[0]
7,ID10055739763,"[0, 241]"
8,ID10074598346,"[0, 598]"
9,ID10081072715,[0]


In [176]:
rv_copy = response_variable_v2.copy()

In [215]:
def get_naive(ID):
    naive = response_variable_v2.loc[response_variable_v2['id'] == ID, 'naive'].values[0]
    return(naive)

In [216]:
get_naive('ID10010854159')

[0, 706]

In [236]:
def get_LTOT_v2(ID):
    try:
        naive = get_naive(ID)
        frame = get_LTOT_frame(ID)

        for day in naive:
            frame_slice = frame.iloc[day:day+180]
            drug_days = np.sum(frame_slice['Has Drug?'])

            if drug_days >= 162:
                return(True)
        return(False)
    except:
        return(np.NaN)

In [239]:
%%time
rv_copy['LTOT_v2'] = rv_copy['id'].map(get_LTOT_v2)

Wall time: 52min 13s


In [253]:
rv_copy

,id,naive,LTOT_v2
0,ID10010854159,"[0, 706]",False
1,ID10013863216,[0],True
2,ID10024447278,[0],True
3,ID1002482139,[0],True
4,ID1003386406,[0],False
5,ID10036289882,[0],False
6,ID10036954381,[0],False
7,ID10055739763,"[0, 241]",True
8,ID10074598346,"[0, 598]",False
9,ID10081072715,[0],False


In [254]:
rv_copy['LTOT_v2'].value_counts()

False    7056
True     6892
Name: LTOT_v2, dtype: int64